## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli
# Trabajo práctico  4: Analisis de los sentimientos en Twitter

# Introducción
Las redes sociales como Twitter han demostrado ser excelentes recursos de información sobre muchos eventos que acontecen en el mundo; tienen el poder de cambiar las opiniones de millones de personas siendo especialmente útil para influir en las masas: campañas políticas, cotización de monedas virtuales, publicidad de ventas, entre otros.
Pensando en esto, se presenta el siguiente objetivo.

# Objetivo:
Analizar los sentimientos con la finalidad de predecir el comportamiento de personas y propagar cambios en tiempo real a medida que se desarrolla el evento que se quiere estudiar.

## Fuente:
Dataset Kaggle: https://www.kaggle.com/code/paoloripamonti/twitter-sentiment-analysis/input



In [ ]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from scipy.stats import mode
import seaborn as sns
import re

# nltk
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline




## Importación de los datos

In [2]:
data = pd.read_csv('./data/twitter.csv', encoding='ANSI')
data.columns = [ 'target', 'id', 'date', 'flag', 'user', 'text']
print(data.shape)
data.head()

## Analisis exploratorio de los datos

In [ ]:
# Check nulls and duplicates
print('Nulls: ', data.isnull().sum())
print('Duplicates: ', data.duplicated().sum())

In [ ]:
# Remove useless columns
data.drop(['id', 'date', 'flag', 'user'], axis=1, inplace=True)

In [ ]:
# Rename target
data['target'] = data['target'].map({0: 'negative', 4: 'positive'})

In [ ]:
# Check target distribution
print('Target distribution: ', data.target.value_counts())

In [ ]:
# Set day of week as category
data['date'] = data['date'].str[:3].astype('category')
data['date'].cat.categories = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
data.head()


In [ ]:
# Get the number of tweets per day and target
data.groupby(['date', 'target']).size()

In [ ]:
# Plot the number of tweets per day and target with a bar chart
# {0: 'Negative', 2: 'Neutral', 4: 'Positive'}
data.groupby(['date', 'target']).size().unstack().plot(kind='bar',  figsize=(10, 5))
plt.title('Number of tweets per day and target')
plt.xlabel('Day of week')
plt.ylabel('Number of tweets')
plt.show()



## Preprocesamiento de texto

### Limpieza del corpus

### Remoción de Stopwords y aplicación de Stemming

In [ ]:
# Expresion regular para eliminar del corpus signos de puntación/ @/ direcciones electronicas #numeros
limpieza_re = "\d+[^0-9]|@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]"

In [ ]:
#Remoción de Stopwords y aplicación de Stemming
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

#Funcion para limpiar el corpus
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(limpieza_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

data.text = data.text.apply(lambda x: preprocess(x))

In [ ]:
data.text

Division de los datos en train y test

In [ ]:
#Asignamos las variables X e Y a modelar
X = data.text
y = data.target

In [ ]:
#Dividimos en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Usamos CountVectorizer para lleva nuestro corpus a una matriz de documentos y términos.
vectorizer=CountVectorizer(strip_accents='unicode'); # hay palabras con acentos por error de tipeo o por ser palabras de otro idioma. Los removemos.
# X_train=vectorizer.fit_transform(X_train);
# X_test=vectorizer.transform(X_test);

Configuración del Pipeline

PARA LEER: el primer pipe es con base a la clase 47_1 clasificacion de textos
pipeline para varios modelos viene de aca https://www.youtube.com/watch?v=ES0lM7QBZnI&list=FLrNGJ7KGOB1XGjvk9OvahOw&index=1
que está buena la idea, y lo que se haria luego es usar CV y tunear los hiperparametros sobre el mejor modelo. 

In [ ]:
#armar un pipeline que incluya la verctorizacion y el clasificador para
#optimizar sobre todos los hiperparámetros a la vez.

#ttps://www.kaggle.com/code/balatmak/text-preprocessing-steps-and-universal-pipeline/notebook 

#https://www.kaggle.com/code/balatmak/text-classification-pipeline-newsgroups20/notebook esta notebook está buenisima

In [ ]:
#ESTO ES UNA BASE 

#iniciamos el modelo
#MNB = MultinomialNB()

In [ ]:
# text_tfidf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer())])
# tfidf_matrix = text_tfidf.fit_transform(texts_all_train)
#https://www.kaggle.com/code/benboyet/web-mining-project-final-prediction/script

In [ ]:
#Código de pipe la verctorizacion y el clasificador para optimizar sobre todos los hiperparámetros a la vez con gridsearch

pasos=[('vect',vectorizer),('classifier',MultinomialNB())]
pipe=Pipeline(pasos)

#Definimos TFIDvectorizer
vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode'); #ngram_range=(1,2)

#definimos la validacion cruzda
skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True)

#parametros del grid         ##usar otros valores
params_grid={'classifier__alpha':[1,2,1.5],'vect__min_df':[1,3]}; #frecuencia menor al dado

#entrenamos el modelo
GS_CV=GridSearchCV(pipe,params_grid,cv=skf,verbose=1,n_jobs=-1);

GS_CV.fit(X_train, y_train);

print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)


In [ ]:
pipe_pred = GS_CV.predict(X_test)

conf_matrix = confusion_matrix(y_test, pipe_pred); 
plt.figure(figsize=(5, 2)); sns.heatmap(conf_matrix,  annot=True, fmt="d");
plt.title("Confusion matrix"); plt.ylabel('True class'); plt.xlabel('Predicted class');plt.show()

In [ ]:
#reporte de clasificación
print(classification_report(y_test, pipe_pred))

In [ ]:
#modelo final con parametros seteados

vectorizer_2=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode', min_df=1);
MNB = MultinomialNB(alpha=2,)
pasos_2=[('vectorizer',vectorizer_2),('MNB',MNB)]

In [ ]:
#configurar pipe
pipe_2 = Pipeline(pasos_2)
pipe_2.fit(X_train, y_train)

In [ ]:
pipe_2_pred= pipe_2.predict(X_test)

In [ ]:
X_test

In [ ]:
#reporte de clasificación
print(classification_report(y_test, pipe_2_pred))

In [ ]:
conf_matrix = confusion_matrix(y_test, pipe_2_pred); 
plt.figure(figsize=(5, 2)); sns.heatmap(conf_matrix,  annot=True, fmt="d");
plt.title("Confusion matrix"); plt.ylabel('True class'); plt.xlabel('Predicted class');plt.show()

In [ ]:
pipe_2_pred= pipe_2.predict(X_test)

"pipeline para varios modelos"

In [ ]:
pipeline_lr = Pipeline([('vect',vectorizer),
                        ('lr_classifier',LogisticRegression())])

In [ ]:
pipeline_knn = Pipeline([('vect',vectorizer),
                        ('knn_classifier',KNeighborsClassifier())])

In [ ]:
#lista de pipelines
pipelines = [pipeline_lr, pipeline_knn]

In [ ]:
best_accuracy=0.0
best_classifier=0
best_pipeline=""

In [ ]:
#diccionario de pipeline y tipos de clasificador para referencia
pipe_dict = {0: "Logistc Regresion", 1:"knn"}

#entrenamiento 
for pipe in pipelines:
    pipe.fit(X_train, y_train)

In [ ]:
for i, model in enumerate(pipelines):
    print("{} Accuracy para el conjunto de prubea: {}".format(pipe_dict[i], model.score(X_test, y_test)))

# ACA AGREGO UN PIPELINE CON GRID SEARCH PARA ARBOL DE DECISIONES CON FEATURE IMPORTANCE

In [ ]:
vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode');

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

pipeline_dt= Pipeline([('vect',vectorizer), ('tree', DecisionTreeClassifier()),('feature_selection', SelectFromModel(DecisionTreeClassifier()))])

# Realizar la búsqueda de Grid Search con validación cruzada
GS_dt=GridSearchCV(pipeline_dt,params_grid,cv=skf,scoring='accuracy', verbose=1,n_jobs=-1)


In [ ]:
# Ajustar el modelo a los datos de entrenamiento
GS_dt.fit(X_train, y_train)


In [ ]:
# Obtener la importancia de las características
feature_importances = GS_dt.best_estimator_.named_steps['tree'].feature_importances_

# Crear un DataFrame para mostrar la importancia de las características
importances = pd.DataFrame({'feature': data.feature_names, 'importance': feature_importances})
importances = importances.sort_values('importance', ascending=False)
print(importances)

# Imprimir los mejores parámetros encontrados por Grid Search
print(GS_dt.best_params_)


### Wordcloud

In [ ]:
#https://www.kaggle.com/code/mskorski/topic-extraction-and-visualization-musk-s-tweets

### Tokenización

### Remoción de stopwords

### Lematización/ Stemming

### Vectorización

### TF-IDF

### Word2Vec

### Singular Value Decomposition 

## Modelos